In [1]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import os
import cv2
# from google.colab.patches import cv2_imshow
from skimage.transform import rescale, resize, downscale_local_mean
import random
from torchvision.models.resnet import resnet50
# import Stream_Resnet
# import collections
# import collections.abc
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
from tqdm import tqdm
import os
import numpy as np
from torchvision import transforms
import random
import matplotlib.pyplot as plt
import cv2
import random
from pycocotools.coco import COCO
import skimage.io as io
import PIL
# from torchvision.models import resnet50
# for the stream resnet
# import Stream_Resnet

# import Self_Attention
# for SSIM LOSS
from pytorch_msssim import ssim
# Get cpu or gpu device for training.
import argparse
# import torch.distributed as dist
import pandas as pd
from matplotlib import cm as CM
from model.layers import Self_Attn
from torch.nn import MaxPool1d
import torch.nn.functional as F

In [2]:
def make_four_layers():
    net = resnet50()
    layer0_name = ['conv1','bn1','relu']
    layer1_name = ['maxpool','layer1']
    layer2_name = ['layer2']
    layer3_name = ['layer3']
    
    layer0 = nn.Sequential()
    layer1 = nn.Sequential()
    layer2 = nn.Sequential()
    layer3 = nn.Sequential()

    for n,c in net.named_children():
        if n in layer0_name:
            layer0.add_module(n,c)
        elif n in layer1_name:
            layer1.add_module(n,c)
        elif n in layer2_name:
            layer2.add_module(n,c)
        elif n in layer3_name:
            layer3 = c
        else:
            break

    return layer0, layer1,layer2,layer3
    

class JDimPool(nn.MaxPool1d):
    def forward(self, input):
        b,n, c, w, h = input.size()
        input = input.view(b, n, c* w * h).permute(0, 2, 1)
        pooled = nn.functional.max_pool1d(
            input,
            self.kernel_size,
            self.stride,
            self.padding,
            self.dilation,
            self.ceil_mode,
            self.return_indices,
        )
        pooled = pooled.permute(0, 2, 1)
        return pooled.view(b,c, w, h)

In [3]:
relu = nn.ReLU()

res_q0, res_q1,res_q2,res_q3 = make_four_layers()
res_r0, res_r1,res_r2,res_r3 = make_four_layers()

sa_q1 = Self_Attn(256, nn.ReLU())
sa_q2 = Self_Attn(512, nn.ReLU())
sa_q3 = Self_Attn(1024, nn.ReLU())
j_maxpool = JDimPool(5,1) # stride is useless


maxpool_r1 = nn.MaxPool2d(4, stride=4, padding=0)
maxpool_r2 = nn.MaxPool2d(2, stride=2, padding=0)
# maxpool_r3 = nn.MaxPool2d(2, stride=2, padding=0)

match_query_conv1 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=1)
match_query_conv2 = nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1)
match_query_conv3 = nn.Conv2d(in_channels=1024, out_channels=256, kernel_size=1)

match_reference_conv1 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=1)
match_reference_conv2 = nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1)
match_reference_conv3 = nn.Conv2d(in_channels=1024, out_channels=256, kernel_size=1)

sum_conv1 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=1)
sum_conv2 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=1)
sum_conv3 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=1)

softmax = nn.Softmax(dim=-1)

transpose_convolution = nn.ConvTranspose2d(
    in_channels=1,
    out_channels=1,
    kernel_size=3,
    stride=2,
    padding=1,
    output_padding = 1
)

In [4]:
q = torch.FloatTensor(5, 3, 256, 256)

In [5]:
r = torch.FloatTensor(5, 5, 3, 64, 64) # b, k, c, w, h

In [6]:
q0 = res_q0(q)
q1 = res_q1(q0)
q2 = res_q2(q1)
q3 = res_q3(q2)
q1.shape, q2.shape, q3.shape

(torch.Size([5, 256, 64, 64]),
 torch.Size([5, 512, 32, 32]),
 torch.Size([5, 1024, 16, 16]))

In [7]:
org_ref_size = r.size()
r = r.view(-1,org_ref_size[-3],org_ref_size[-2],org_ref_size[-1])
org_ref_size, r.shape

(torch.Size([5, 5, 3, 64, 64]), torch.Size([25, 3, 64, 64]))

In [8]:
r0 = res_r0(r)
r1 = res_r1(r0)
r2 = res_r2(r1)
r3 = res_r3(r2)
r1.shape, r2.shape, r3.shape

(torch.Size([25, 256, 16, 16]),
 torch.Size([25, 512, 8, 8]),
 torch.Size([25, 1024, 4, 4]))

In [9]:
r1_size = r1.size()
r2_size = r2.size()
r3_size = r3.size()
r1 = r1.view(org_ref_size[0],org_ref_size[1],r1_size[-3],r1_size[-2],r1_size[-1])
r2 = r2.view(org_ref_size[0],org_ref_size[1],r2_size[-3],r2_size[-2],r2_size[-1])
r3 = r3.view(org_ref_size[0],org_ref_size[1],r3_size[-3],r3_size[-2],r3_size[-1])
r1.shape, r2.shape, r3.shape

(torch.Size([5, 5, 256, 16, 16]),
 torch.Size([5, 5, 512, 8, 8]),
 torch.Size([5, 5, 1024, 4, 4]))

In [10]:
r1 = j_maxpool(r1)
r2 = j_maxpool(r2)
r3 = j_maxpool(r3)
r1.shape, r2.shape, r3.shape

(torch.Size([5, 256, 16, 16]),
 torch.Size([5, 512, 8, 8]),
 torch.Size([5, 1024, 4, 4]))

In [11]:
input1, _ = sa_q1(q1)
input2, _ = sa_q2(q2)
input3, _ = sa_q3(q3)
input1.shape, input2.shape, input3.shape

(torch.Size([5, 256, 64, 64]),
 torch.Size([5, 512, 32, 32]),
 torch.Size([5, 1024, 16, 16]))

In [12]:
input1 = match_query_conv1(input1)
input2 = match_query_conv2(input2)
input3 = match_query_conv3(input3)
input1.shape, input2.shape, input3.shape

(torch.Size([5, 256, 64, 64]),
 torch.Size([5, 256, 32, 32]),
 torch.Size([5, 256, 16, 16]))

In [13]:
kernel1 = maxpool_r1(r1)
kernel2 = maxpool_r2(r2)
kernel3 = r3
kernel1.shape, kernel2.shape, kernel3.shape

(torch.Size([5, 256, 4, 4]),
 torch.Size([5, 512, 4, 4]),
 torch.Size([5, 1024, 4, 4]))

In [14]:
kernel1 = match_reference_conv1(kernel1)
kernel2 = match_reference_conv2(kernel2)
kernel3 = match_reference_conv3(kernel3)
kernel1.shape, kernel2.shape, kernel3.shape

(torch.Size([5, 256, 4, 4]),
 torch.Size([5, 256, 4, 4]),
 torch.Size([5, 256, 4, 4]))

In [15]:
M1 = []
M2 = []
M3 = []
for i in range(kernel1.size(0)):
    tmp_m1 = nn.functional.conv2d(input1[i:i+1], kernel1[i:i+1],padding=1,stride=2)
    tmp_m2 = nn.functional.conv2d(input2[i:i+1], kernel2[i:i+1],padding=1,stride=2)
    tmp_m3 = nn.functional.conv2d(input3[i:i+1], kernel3[i:i+1],padding=1,stride=2)
    M1.append(tmp_m1)
    M2.append(tmp_m2)
    M3.append(tmp_m3)

M1 = torch.cat(M1,0)
M2 = torch.cat(M2,0)
M3 = torch.cat(M3,0)

M1.shape, M2.shape, M3.shape

(torch.Size([5, 1, 32, 32]),
 torch.Size([5, 1, 16, 16]),
 torch.Size([5, 1, 8, 8]))

In [16]:
# M1 = nn.functional.max_pool2d(M1,2)

M2 = nn.functional.interpolate(M2,scale_factor=2)
M3 = nn.functional.interpolate(M3,scale_factor=4)
M2.shape, M2.shape

(torch.Size([5, 1, 32, 32]), torch.Size([5, 1, 32, 32]))

In [17]:
S1 = sum_conv1(M1)
S2 = sum_conv2(M2)
S3 = sum_conv3(M3)
S1.shape, S2.shape, S3.shape

(torch.Size([5, 1, 32, 32]),
 torch.Size([5, 1, 32, 32]),
 torch.Size([5, 1, 32, 32]))

In [18]:
S1_W = []
S2_W = []
S3_W = []

In [19]:
for k in range(S1.shape[0]):
    S1_W.append(torch.sum(S1[k]))
    S2_W.append(torch.sum(S2[k]))
    S3_W.append(torch.sum(S3[k]))

In [20]:
S1_W = torch.stack(S1_W)
S2_W = torch.stack(S2_W)
S3_W = torch.stack(S3_W)
S1_W.shape, S2_W.shape, S3_W.shape

(torch.Size([5]), torch.Size([5]), torch.Size([5]))

In [21]:
S1_W.shape[0]

5

In [22]:
W = []

In [23]:
for k in range(S1_W.shape[0]):
    W.append(softmax(torch.tensor([S1_W[k], S2_W[k], S3_W[k]])))
    print(W[k])

tensor([0., 0., 1.])
tensor([0., 0., 1.])
tensor([0., 0., 1.])
tensor([0., 0., 1.])
tensor([0., 0., 1.])


In [24]:
FS = []

In [25]:
for k in range(M1.shape[0]):
    FS.append(W[k][0] * M1[k] + W[k][1] * M2[k] + W[k][2] * M3[k])
    print(FS[k].shape)

torch.Size([1, 32, 32])
torch.Size([1, 32, 32])
torch.Size([1, 32, 32])
torch.Size([1, 32, 32])
torch.Size([1, 32, 32])


In [26]:
FS = torch.stack(FS)
FS.shape

torch.Size([5, 1, 32, 32])

In [27]:
FS = transpose_convolution(FS)
FS.shape

torch.Size([5, 1, 64, 64])

In [28]:

FS = nn.functional.interpolate(FS,
                scale_factor=4,
                mode='bilinear',
                align_corners=True)
FS.shape

torch.Size([5, 1, 256, 256])